In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [2]:
data = pd.read_csv('CC2020_train_final.csv')

In [3]:
data.head(3)

,YEAR,LOCATION,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,18,Loc 5608,Inbred_142,Cluster11,Tester_740,Cluster10,1.135462
1,18,Loc 4533,Inbred_142,Cluster11,Tester_740,Cluster10,1.139813
2,18,Loc 5620,Inbred_142,Cluster11,Tester_740,Cluster10,1.117778


In [4]:
X = data.drop(['YEAR', 'LOCATION'], axis=1)

In [5]:
len(X)

199476

In [6]:
clean_data = X.groupby(['INBRED', 'INBRED_CLUSTER','TESTER', 'TESTER_CLUSTER' ],as_index=False).mean()

In [7]:
clean_data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,Inbred_1071,Cluster8,Tester_1345,Cluster1,0.986544
1,Inbred_1071,Cluster8,Tester_4373,Cluster3,1.057704
2,Inbred_1071,Cluster8,Tester_4473,Cluster5,1.023704
3,Inbred_1071,Cluster8,Tester_4541,Cluster4,1.014735
4,Inbred_1071,Cluster8,Tester_5305,Cluster4,1.062727


In [8]:
len(clean_data)

10919

In [9]:
y = clean_data.YIELD

In [10]:
data = clean_data.drop(['YIELD'], axis=1)

In [11]:
data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,Inbred_1071,Cluster8,Tester_1345,Cluster1
1,Inbred_1071,Cluster8,Tester_4373,Cluster3
2,Inbred_1071,Cluster8,Tester_4473,Cluster5
3,Inbred_1071,Cluster8,Tester_4541,Cluster4
4,Inbred_1071,Cluster8,Tester_5305,Cluster4


In [12]:
X = data

In [13]:
label_enc = preprocessing.LabelEncoder()

In [14]:
X['INBRED'] = label_enc.fit_transform(X['INBRED'])
X['INBRED_CLUSTER'] = label_enc.fit_transform(X['INBRED_CLUSTER'])
X['TESTER'] = label_enc.fit_transform(X['TESTER'])
X['TESTER_CLUSTER'] = label_enc.fit_transform(X['TESTER_CLUSTER'])

In [15]:
X.head(3)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,0,12,0,0
1,0,12,125,7
2,0,12,136,9


In [16]:
#notice how these are all same.. number varies within same location or year

In [17]:
enc = preprocessing.OneHotEncoder()

In [18]:
X = enc.fit_transform(X)

#Classifiers

In [19]:
#GradientBoosting Basic

In [23]:
fit_intercept= [True, False]
precompute= [False, True ]
normalize= [False, True ]
# Number of trees in random forest
alpha = [0.05, 0.25, 0.5, 1, 1.5]
# Number of features to consider at every split
max_iter = [int(x) for x in np.linspace(500, 2000, num = 10)]
# Maximum number of levels in tree
l1_ratio = [0.1, 0.5, 1, 1.5]
tol = [0.0001, 1, 0.005]
warm_start = [True, False]
copy_X= [True, False]
positive = [True, False]
selection = ['random', 'cyclic']

# Create the random grid
random_grid = {'fit_intercept': fit_intercept,
               'precompute': precompute,
               'normalize': normalize,
               'alpha': alpha,
               'max_iter': max_iter,
               'l1_ratio': l1_ratio,
                'tol': tol,
                'warm_start': warm_start,
               'copy_X': copy_X,
               'positive': positive,
               'selection': selection,
              }
print(random_grid)

{'fit_intercept': [True, False], 'precompute': [False, True], 'normalize': [False, True], 'alpha': [0.05, 0.25, 0.5, 1, 1.5], 'max_iter': [500, 666, 833, 1000, 1166, 1333, 1500, 1666, 1833, 2000], 'l1_ratio': [0.1, 0.5, 1, 1.5], 'tol': [0.0001, 1, 0.005], 'warm_start': [True, False], 'copy_X': [True, False], 'positive': [True, False], 'selection': ['random', 'cyclic']}


In [24]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
ENet = ElasticNet()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
enet_random = RandomizedSearchCV(estimator = ENet, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42)
# Fit the random search model
enet_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] warm_start=False, tol=1, selection=random, precompute=False, positive=True, normalize=True, max_iter=1166, l1_ratio=0.1, fit_intercept=True, copy_X=True, alpha=0.25 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  warm_start=False, tol=1, selection=random, precompute=False, positive=True, normalize=True, max_iter=1166, l1_ratio=0.1, fit_intercept=True, copy_X=True, alpha=0.25, total=   0.2s
[CV] warm_start=False, tol=1, selection=random, precompute=False, positive=True, normalize=True, max_iter=1166, l1_ratio=0.1, fit_intercept=True, copy_X=True, alpha=0.25 
[CV]  warm_start=False, tol=1, selection=random, precompute=False, positive=True, normalize=True, max_iter=1166, l1_ratio=0.1, fit_intercept=True, copy_X=True, alpha=0.25, total=   0.0s
[CV] warm_start=False, tol=1, selection=random, precompute=False, positive=True, normalize=True, max_iter=1166, l1_ratio=0.1, fit_intercept=True, copy_X=True, alpha=0.25 
[CV]  warm_start=False, tol=1, selection=random, precompute=False, positive=True, normalize=True, max_iter=1166, l1_ratio=0.1, fit_intercept=True, copy_X=True, alpha=0.25, total=   0.0s
[CV] warm_start=True, tol=1, selection=cyclic, precompute=True, positive=False, normalize=True, max_

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s



[CV] warm_start=True, tol=0.0001, selection=cyclic, precompute=True, positive=False, normalize=False, max_iter=1666, l1_ratio=1, fit_intercept=False, copy_X=True, alpha=0.5 
[CV]  warm_start=True, tol=0.0001, selection=cyclic, precompute=True, positive=False, normalize=False, max_iter=1666, l1_ratio=1, fit_intercept=False, copy_X=True, alpha=0.5, total=   0.0s
[CV] warm_start=True, tol=0.0001, selection=cyclic, precompute=True, positive=False, normalize=False, max_iter=1666, l1_ratio=1, fit_intercept=False, copy_X=True, alpha=0.5 
[CV]  warm_start=True, tol=0.0001, selection=cyclic, precompute=True, positive=False, normalize=False, max_iter=1666, l1_ratio=1, fit_intercept=False, copy_X=True, alpha=0.5, total=   0.0s
[CV] warm_start=False, tol=0.0001, selection=cyclic, precompute=True, positive=True, normalize=True, max_iter=2000, l1_ratio=0.5, fit_intercept=False, copy_X=False, alpha=0.5 
[CV]  warm_start=False, tol=0.0001, selection=cyclic, precompute=True, positive=True, normalize=T

...

[CV]  warm_start=True, tol=0.0001, selection=random, precompute=False, positive=True, normalize=False, max_iter=833, l1_ratio=0.5, fit_intercept=True, copy_X=True, alpha=1.5, total=   0.0s
[CV] warm_start=True, tol=0.0001, selection=random, precompute=False, positive=True, normalize=False, max_iter=833, l1_ratio=0.5, fit_intercept=True, copy_X=True, alpha=1.5 
[CV]  warm_start=True, tol=0.0001, selection=random, precompute=False, positive=True, normalize=False, max_iter=833, l1_ratio=0.5, fit_intercept=True, copy_X=True, alpha=1.5, total=   0.0s
[CV] warm_start=False, tol=1, selection=random, precompute=False, positive=False, normalize=True, max_iter=1000, l1_ratio=1, fit_intercept=False, copy_X=True, alpha=1.5 
[CV]  warm_start=False, tol=1, selection=random, precompute=False, positive=False, normalize=True, max_iter=1000, l1_ratio=1, fit_intercept=False, copy_X=True, alpha=1.5, total=   0.0s
[CV] warm_start=False, tol=1, selection=random, precompute=False, positive=False, normalize=T

[CV]  warm_start=True, tol=1, selection=cyclic, precompute=True, positive=True, normalize=True, max_iter=833, l1_ratio=0.1, fit_intercept=True, copy_X=False, alpha=0.5, total=   0.0s
[CV] warm_start=True, tol=1, selection=cyclic, precompute=True, positive=True, normalize=True, max_iter=833, l1_ratio=0.1, fit_intercept=True, copy_X=False, alpha=0.5 
[CV]  warm_start=True, tol=1, selection=cyclic, precompute=True, positive=True, normalize=True, max_iter=833, l1_ratio=0.1, fit_intercept=True, copy_X=False, alpha=0.5, total=   0.0s
[CV] warm_start=False, tol=1, selection=cyclic, precompute=False, positive=True, normalize=False, max_iter=1666, l1_ratio=1.5, fit_intercept=True, copy_X=True, alpha=0.05 
[CV]  warm_start=False, tol=1, selection=cyclic, precompute=False, positive=True, normalize=False, max_iter=1666, l1_ratio=1.5, fit_intercept=True, copy_X=True, alpha=0.05, total=   0.0s
[CV] warm_start=False, tol=1, selection=cyclic, precompute=False, positive=True, normalize=False, max_iter=

[CV]  warm_start=False, tol=0.005, selection=cyclic, precompute=True, positive=True, normalize=True, max_iter=2000, l1_ratio=0.5, fit_intercept=True, copy_X=True, alpha=1.5, total=   0.0s
[CV] warm_start=False, tol=1, selection=cyclic, precompute=True, positive=True, normalize=False, max_iter=1166, l1_ratio=1.5, fit_intercept=True, copy_X=False, alpha=0.05 
[CV]  warm_start=False, tol=1, selection=cyclic, precompute=True, positive=True, normalize=False, max_iter=1166, l1_ratio=1.5, fit_intercept=True, copy_X=False, alpha=0.05, total=   0.0s
[CV] warm_start=False, tol=1, selection=cyclic, precompute=True, positive=True, normalize=False, max_iter=1166, l1_ratio=1.5, fit_intercept=True, copy_X=False, alpha=0.05 
[CV]  warm_start=False, tol=1, selection=cyclic, precompute=True, positive=True, normalize=False, max_iter=1166, l1_ratio=1.5, fit_intercept=True, copy_X=False, alpha=0.05, total=   0.0s
[CV] warm_start=False, tol=1, selection=cyclic, precompute=True, positive=True, normalize=Fals

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    3.9s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=ElasticNet(alpha=1.0, copy_X=True,
                                        fit_intercept=True, l1_ratio=0.5,
                                        max_iter=1000, normalize=False,
                                        positive=False, precompute=False,
                                        random_state=None, selection='cyclic',
                                        tol=0.0001, warm_start=False),
                   iid='deprecated', n_iter=100, n_jobs=None,
                   param_distributions={'alpha': [0.05, 0.25, 0.5, 1, 1.5],
                                        'copy_X': [True, F...
                                        'fit_intercept': [True, False],
                                        'l1_ratio': [0.1, 0.5, 1, 1.5],
                                        'max_iter': [500, 666, 833, 1000, 1166,
                                                     1333, 1500, 1666, 1833,
                       

In [25]:
enet_random.best_params_

{'warm_start': False,
 'tol': 1,
 'selection': 'random',
 'precompute': False,
 'positive': True,
 'normalize': True,
 'max_iter': 1166,
 'l1_ratio': 0.1,
 'fit_intercept': True,
 'copy_X': True,
 'alpha': 0.25}

In [37]:
scores = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=np.random)
    
    my_model = ElasticNet(tol = 1, selection = 'cyclic', positive = 'True', normalize = 'True', max_iter = 11660, l1_ratio = 0.4, alpha = 0.55)
    my_model.fit(X_train, y_train)
    predictions = my_model.predict(X_test)
    score = mean_absolute_error(predictions, y_test)
    scores.append(score)

    
print('Total CV MSE is {}'.format(np.mean(scores)))
print('Total STD is {}'.format(np.std(scores)))

Total CV MSE is 0.044407526764649555
Total STD is 0.0004907437583704168


In [20]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [100,110,120],
    'max_features': max_features,
    'min_samples_leaf': [1,2,3],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [600, 800, 1000]
}




In [27]:
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, verbose = 2, n_jobs = -1)

In [28]:
# Fit the grid search to the data
grid_search.fit(X, y)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 23.5min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed: 25.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [24]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 120,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 12,
 'n_estimators': 1000}

In [33]:
scores = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=np.random)
    my_model = RandomForestRegressor(n_estimators= 1000, min_samples_split=12,min_samples_leaf = 1, max_features='sqrt', 
                                     max_depth =130, bootstrap=True)
    my_model.fit(X_train, y_train)
    predictions = my_model.predict(X_test)
    score = mean_absolute_error(predictions, y_test)
    scores.append(score)

    
print('Total CV MSE is {}'.format(np.mean(scores)))
print('Total STD is {}'.format(np.std(scores)))

Total CV MSE is 0.040378190507865556
Total STD is 0.0004462042931164293


In [26]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [110,120,130],
    'max_features': max_features,
    'min_samples_split': [10, 12, 14],
    'n_estimators': [800, 1000, 1200]
}


In [29]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 130,
 'max_features': 'sqrt',
 'min_samples_split': 10,
 'n_estimators': 1000}